In [93]:
import pandas as pd
import numpy as np
import csv
from zipfile import ZipFile
import io
import requests
import tabula
import re

### INFLATION

In [94]:
#defining the file_path
file_path = 'Datasets_ECON323/Inflation.csv'

# Read in the dataset 
inflation = pd.read_csv(file_path)

#drop columns 
columns_to_drop = ['INDINF_CPI_TRIM_Q', 'INDINF_CPI_MEDIAN_Q', 'INDINF_CPI_Q']
inflation.drop(columns=columns_to_drop, inplace=True)

#drop observations by index
rows_to_drop = inflation.iloc[0:68]
inflation.drop(index=rows_to_drop.index, inplace=True)

# Rename a single column
inflation.rename(columns={'INDINF_CPI_COMMON_Q': 'Inflation (CPI)', 'date':'Year'}, inplace=True)

#need to drop Q1-3 data and retain Q4 data using Boolean indexing    

# Create a boolean mask
mask = (np.arange(len(inflation)) - 3) % 4 == 0

# Apply the mask to keep every 4th row starting from the second
filtered_inflation = inflation[mask]

#print(filtered_inflation)

#need to remove "Q4" and turn the data column into a time object 
#filtered_inflation.loc[:, 'Year'] = filtered_inflation['Year'].str.replace('Q4', '')
filtered_inflation.loc[:,'Year']=pd.to_datetime(filtered_inflation['Year'].str.replace('Q4', ''),format='%Y').dt.year
filtered_inflation = filtered_inflation.set_index('Year')

### Foreign Currency Reserves 

In [95]:
#defining the file_path

file_path = 'Datasets_ECON323/Foreign_Currency_Reserves.csv'

#read in the dataset
FCR = pd.read_csv(file_path) 

#drop irrelevant columns
columns_to_drop_FCR = ['Type of reserve', 'GEO', 'UOM','DGUID', 'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID','VECTOR', 'COORDINATE','STATUS', 'TERMINATED', 'DECIMALS', 'SYMBOL'] 
FCR.drop(columns = columns_to_drop_FCR, inplace = True) 

#dropping the observations with variables that are not of interest using indexes
FCR.drop(index=range(159, 973), inplace=True)

#filter to keep the last month of each year using Boolean indexing 
mask = (np.arange(len(FCR)) - 14) % 12 == 0

# Apply the mask to keep every 12th row starting from the second
filtered_FCR = FCR[mask]

filtered_FCR.loc[:,'REF_DATE']=pd.to_datetime(filtered_FCR['REF_DATE']).dt.year
filtered_FCR = filtered_FCR.rename(columns={'REF_DATE':'Year','VALUE':'CAN Official International Reserves'})
filtered_FCR = filtered_FCR.set_index('Year')

### Foreign Direct Investment

In [96]:
#defining the file_path
file_path = "Datasets_ECON323/FDI.csv"

#read in the dataset
FDI = pd.read_csv(file_path)

#drop irrelevant columns 
columns_to_drop_FDI = ['GEO', 'UOM','DGUID', 'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID','VECTOR',
                       'COORDINATE','STATUS', 'TERMINATED', 'DECIMALS', 'SYMBOL',
                       'Current account and capital account', 'Countries or regions','Receipts, payments and balances']
FDI.drop(columns = columns_to_drop_FDI, inplace = True)

#dropping the observations with variables that are not of interest using indexes
FDI.drop(index=range(0, 392), inplace=True)
FDI.drop(index=range(406, 588), inplace=True)

#print to check

FDI.loc[:,'REF_DATE']=pd.to_datetime(FDI['REF_DATE'],format='%Y').dt.year
FDI = FDI.rename(columns={'REF_DATE':'Year','VALUE':'FDI Balanaces'})
FDI = FDI.set_index('Year')

In [97]:
#filtered_FCR
#FDI
#filtered_inflation.head()

### Dairy Industry Supply

In [98]:
file_path = "Datasets_ECON323/dairysupply.csv"
df = pd.read_csv(file_path, parse_dates=["REF_DATE"])

In [99]:
df_yearly = df.groupby([df['Supply and disposition'], df['REF_DATE'].dt.year])['VALUE'].sum().reset_index() #sum commodities
df_yearly = df_yearly.rename(columns={"Supply and disposition": "Category", "REF_DATE": "Year", "VALUE":"Value"}) #rename col
df_yearly['Category']=df_yearly['Category'].replace('Supply, total', 'Total Dairy Supply') #rename values
df_yearly = df_yearly.set_index('Year') #index by year

dairy_ind = df_yearly.pivot_table(
    index='Year',
    columns='Category',
    values='Value'
)
dairy_ind = dairy_ind.rename(columns={'Exports':'Total Dairy Exports','Imports':'Total Dairy Imports'})
#dairy_ind.head()

### Grain Industry Supply

In [100]:
file_path = "Datasets_ECON323/grainsupply.csv"
cdf = pd.read_csv(file_path,parse_dates=['REF_DATE'])

In [101]:
#all grains
cdfy = cdf.groupby([cdf['Supply and disposition of grains'], cdf['REF_DATE'].dt.year])['VALUE'].sum().reset_index() 
cdfy = cdfy.rename(columns={"Supply and disposition of grains": "Category", "REF_DATE": "Year", "VALUE":"Value"})
cdfy['Category']=cdfy['Category'].replace({'Imports':'Total Grain Imports', 'Total exports':'Total Grain Exports','Total supplies':'Total Grain Supply'})
cdfy = cdfy.set_index('Year')

grain_ind = cdfy.pivot_table(
    index='Year',
    columns='Category',
    values='Value'
)
#grain_ind.head()

### Exchange rate

In [102]:
file_path = "Datasets_ECON323/CADMXN.csv"
cad_mxn = pd.read_csv(file_path, index_col=0)
file_path = "Datasets_ECON323/CADUSD.csv"
cad_usd = pd.read_csv(file_path, index_col=0)

In [103]:
cad_mxn.reset_index(inplace=True)
cad_usd.reset_index(inplace=True)

In [104]:
cad_mxn['Date'] = pd.to_datetime(cad_mxn['Date'])
cad_mxn = cad_mxn[(cad_mxn['Date'] >= '2010-01-01') & (cad_mxn['Date'] <= '2023-12-31')]

# Resample data annually and calculate the mean
cad_mxn_annual = cad_mxn.resample('YE', on='Date').mean()

# Calculate percentage change in exchange rates
cad_mxn_annual['Percentage Change'] = ((cad_mxn_annual['Close'] - cad_mxn_annual['Open']) / cad_mxn_annual['Open']) * 100

#Extract year from index and create a new column
cad_mxn_annual['Year'] = cad_mxn_annual.index.year

#Filter data for the years 2010-2023
cad_mxn_annual = cad_mxn_annual[(cad_mxn_annual.index.year >= 2010) & (cad_mxn_annual.index.year <= 2023)]
cad_mxn_annual = cad_mxn_annual.set_index('Year')
cad_mxn_annual = cad_mxn_annual.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'])

In [105]:
cad_usd['Date'] = pd.to_datetime(cad_usd['Date'])
cad_usd = cad_usd[(cad_usd['Date'] >= '2010-01-01') & (cad_usd['Date'] <= '2023-12-31')]

# Resample data annually and calculate the mean
cad_usd_annual = cad_usd.resample('YE', on='Date').mean()

#Calculate percentage change in exchange rates 
cad_usd_annual['Percentage Change'] = ((cad_usd_annual['Close'] - cad_usd_annual['Open']) / cad_usd_annual['Open']) * 100

#Extract year from index and create a new column
cad_usd_annual['Year'] = cad_usd_annual.index.year

#Filter data for the years 2010-2023
cad_usd_annual = cad_usd_annual[(cad_usd_annual.index.year >= 2010) & (cad_usd_annual.index.year <= 2023)]
cad_usd_annual = cad_usd_annual.set_index('Year')
cad_usd_annual = cad_usd_annual.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'])

In [106]:
merged_ex = cad_mxn_annual.merge(cad_usd_annual, on='Year', suffixes=('_cad_mxn', '_cad_usd'))
#merged_ex = merged_ex.drop(columns=['level_0_cad_mxn','date','level_0_cad_usd','index'])
# Display the merged DataFrame
merged_ex = merged_ex.rename(columns={'Adj Close_cad_mxn':'CAD-MXN EX',
                                      'Percentage Change_cad_mxn':'Percent Change CAD-MXN',
                                      'Adj Close_cad_usd':'CAD-USD EX',
                                      'Percentage Change_cad_usd':'Percent Change CAD-USD'})
#merged_ex

### Revenues

Dairy

In [107]:
# in billions USD

data = {
    'Year': [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
    'Dairy Product Manufacturing': [10.74, 10.47, 10.74, 10.47, 10.43, 10.30, 11.26, 11.55, 12.22, 12.54, 12.58, 12.89, 13.25, 13.66, 13.91],
    'Ice Cream and Frozen Dessert Manufacturing': [0.53, 0.53, 0.53, 0.53, 0.52, 0.53, 0.53, 0.63, 0.70, 0.74, 0.77, 0.80, 0.84, 0.89, 0.92]
}

# Create a DataFrame
dairy_revenue = pd.DataFrame(data)

# Use backfill to fill in missing values
dairy_revenue = dairy_revenue.bfill()

# Add new rows for the years 2010 and 2011 and fill them using backfill
new_rows = pd.DataFrame({'Year': [2010, 2011], 'Dairy Product Manufacturing': None, 'Ice Cream and Frozen Dessert Manufacturing': None})
dairy_revenue = pd.concat([new_rows, dairy_revenue]).sort_values(by='Year').reset_index(drop=True)
dairy_revenue = dairy_revenue.bfill()

# Sum the values of 'Dairy Product Manufacturing' and 'Ice Cream and Frozen Dessert Manufacturing' columns
dairy_revenue['Total Dairy Manufacturing'] = dairy_revenue['Dairy Product Manufacturing'] + dairy_revenue['Ice Cream and Frozen Dessert Manufacturing']

dairy_revenue = dairy_revenue.set_index('Year')
dairy_revenue = dairy_revenue.drop(columns={'Dairy Product Manufacturing','Ice Cream and Frozen Dessert Manufacturing'},index=2024)
dairy_revenue = dairy_revenue[~dairy_revenue.index.duplicated(keep='first')]
#dairy_revenue

C:\Users\Arya Leong\AppData\Local\Temp\ipykernel_7436\2270627293.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dairy_revenue = pd.concat([new_rows, dairy_revenue]).sort_values(by='Year').reset_index(drop=True)


Wheat

In [108]:
file_path = 'Datasets_ECON323/wheat_prod.csv'
wt_prod = pd.read_csv(file_path, low_memory=False, parse_dates=['REF_DATE'])

wt_prod = wt_prod.drop(columns=['GEO', 'DGUID', 'Type of crop', 
                           'Supply and disposition of grains', 'UOM', 
                           'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR',
                           'COORDINATE', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS'])
wt_prod = wt_prod.groupby(wt_prod['REF_DATE'].dt.year)['VALUE'].sum()
wt_prod = wt_prod.to_frame().reset_index()
wt_prod = wt_prod.rename(columns={'REF_DATE': 'Year',
                                  'VALUE': 'Total Wheat Production'})

file_path = 'Datasets_ECON323/wheat_price.csv'
wt_price = pd.read_csv(file_path, low_memory=False, parse_dates=['date'], dayfirst=True)
# wt_price.info()
wt_price = wt_price.rename(columns={'date': 'Year', ' value': 'Wheat Price'})
wt_price = wt_price.groupby(wt_price['Year'].dt.year)['Wheat Price'].mean()
wt_price = wt_price.to_frame().reset_index()
wt_price = wt_price[wt_price['Year'] > 2009]
# bushel * bushel price 
wt_price['Wheat Price'] = wt_price['Wheat Price'] * 36.7437
wt_rev = pd.merge(wt_prod, wt_price[['Year', 'Wheat Price']], on='Year', how='left')
wt_rev['Wheat Market Value'] = wt_rev['Total Wheat Production'] * wt_rev['Wheat Price']
wt_rev = wt_rev.set_index('Year')
#wt_rev

### CAN Pop

In [109]:
file_path = 'Datasets_ECON323/canadapopulation.csv'
ca_pop = pd.read_csv(file_path, low_memory=False)
ca_pop = ca_pop.drop(ca_pop.index[0:3])
ca_pop = ca_pop.drop(columns='World Development Indicators')
ca_pop = ca_pop.T
ca_pop.reset_index(inplace=True)
ca_pop.columns = ca_pop.iloc[0]
ca_pop = ca_pop.drop(ca_pop.index[0:1])
ca_pop = ca_pop.rename(columns={'Data Source': 'Year'})

year_values = ca_pop['Year'].tolist()

year = 1960

for i, value in enumerate(year_values):
    if value.startswith('Unnamed: '):
        year_values[i] = year
        year += 1
        
ca_pop['Year'] = year_values

columns_to_keep = ['Year', 'Canada']
columns_to_drop = [col for col in ca_pop.columns if col not in columns_to_keep]
ca_pop = ca_pop.drop(columns_to_drop, axis=1)
ca_pop = ca_pop.ffill()
ca_pop = ca_pop[ca_pop['Year'] > 2009]
ca_pop.reset_index(inplace=True)
ca_pop = ca_pop.drop(columns='index')

ca_pop = ca_pop.set_index('Year')
ca_pop = ca_pop.rename(columns={'Canada':'CAN Population'})

C:\Users\Arya Leong\AppData\Local\Temp\ipykernel_7436\2445097767.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ca_pop = ca_pop.ffill()


### CAN GDP

In [110]:
file_path = 'Datasets_ECON323/canadagdp.xls'
ca_gdp = pd.read_excel(file_path)

# Cleaning the dataset
ca_gdp = ca_gdp.drop(columns='World Development Indicators')
ca_gdp = ca_gdp.drop(ca_gdp.index[:3])
ca_gdp = ca_gdp.T
ca_gdp.reset_index(inplace=True)
ca_gdp.columns = ca_gdp.iloc[0]
ca_gdp = ca_gdp.drop(ca_gdp.index[0:3])
ca_gdp = ca_gdp.rename(columns={'Data Source': 'Year'})

# We want to rename the unnamed values in the 'Year' columns to the corresponding years
year = 1960
year_values = ca_gdp['Year'].tolist()

for i, value in enumerate(year_values):
    if value.startswith('Unnamed: '):
        year_values[i] = year
        year += 1
        
ca_gdp['Year'] = year_values

# We want to clean the dataframe so that it only has the 'Year' and 'Canada' columns
columns_to_keep = ['Year', 'Canada']
columns_to_drop = [col for col in ca_gdp.columns if col not in columns_to_keep]

ca_gdp = ca_gdp.drop(columns_to_drop, axis=1)
ca_gdp = ca_gdp[ca_gdp['Year'] > 2009]
ca_gdp.reset_index(inplace=True)
ca_gdp = ca_gdp.drop(columns='index')
ca_gdp = ca_gdp.set_index('Year')
ca_gdp.loc[2023,'Canada']=2145400000000.00000
ca_gdp = ca_gdp.rename(columns={'Canada':'CAN GDP'})
#ca_gdp

# Full Combined

In [111]:
combined_df = pd.concat([ca_pop, ca_gdp, FDI, filtered_FCR, merged_ex, filtered_inflation, dairy_ind, grain_ind, dairy_revenue, wt_rev],axis=1)
combined_df

,CAN Population,CAN GDP,FDI Balanaces,CAN Official International Reserves,CAD-MXN EX,Percent Change CAD-MXN,CAD-USD EX,Percent Change CAD-USD,Inflation (CPI),Total Dairy Exports,Total Dairy Imports,Total Dairy Supply,Total Grain Exports,Total Grain Imports,Total Grain Supply,Total Dairy Manufacturing,Total Wheat Production,Wheat Price,Wheat Market Value
Year,,,,,,,,,,,,,,,,,,,
2010,34004889.0,1617343367486.258545,-59351,57151,12.227417,-0.029978,0.966085,0.378777,1.1,16375.0,32548.0,2105430.0,114282.6,1928.4,326761.5,11.27,77199.4,215.730063,1.665423e+07
2011,34339328.0,1793326630174.518555,-48112,65819,12.625667,0.778917,1.012856,-0.405890,2.0,19626.0,38004.0,1996011.0,109721.4,1565.5,316423.1,11.00,71887.2,262.506325,1.887084e+07
2012,34714222.0,1828366481521.595215,-64579,68546,13.140333,-0.485945,1.001247,0.244768,1.6,22537.0,36888.0,2114434.0,118961.9,1329.1,322104.6,11.27,77822.0,277.092990,2.156393e+07
2013,35082954.0,1846597421834.983398,-59759,71937,12.410317,-0.340288,0.966006,-0.598779,1.1,28473.0,35780.0,2247301.0,127091.2,1297.5,348363.2,11.00,92081.1,252.092620,2.321297e+07
2014,35437435.0,1805749878439.941162,-46278,74700,12.057342,0.319983,0.902780,-0.690286,1.7,24758.0,41506.0,2137260.0,143813.8,1227.2,395115.6,10.95,104620.3,216.448529,2.264491e+07
2015,35702908.0,1556508816217.140137,-69569,79753,12.396437,-0.012515,0.776641,-1.421982,2.1,26739.0,42070.0,1995963.0,150458.2,1585.4,373909.5,10.83,86531.6,186.789078,1.616316e+07
2016,36109487.0,1527994741907.425049,-62553,82718,14.205670,2.168690,0.757134,0.316442,1.3,36676.0,53776.0,1960918.0,143811.1,1757.5,367288.8,11.79,87434.7,161.468586,1.411796e+07
2017,36545236.0,1649265644244.094971,-59998,86625,14.542284,-0.056275,0.772483,0.573924,1.4,85096.0,48433.0,2911364.0,142094.0,2014.0,385372.3,12.18,94657.1,161.723297,1.530826e+07
2018,37065084.0,1725329192783.023926,-53141,83926,14.783048,-0.794437,0.769920,-0.699336,1.8,76427.0,52477.0,2923189.0,150235.5,2354.5,391989.8,12.92,93106.5,182.824768,1.702217e+07
